<a href="https://colab.research.google.com/github/TayTechie07/transformer-based-fake-news-detection/blob/main/FakeNewsDetection_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch


In [2]:
df = pd.read_csv("WELFake_Dataset.csv", header=None, engine="python",
    encoding="latin-1",
    on_bad_lines="skip")


In [3]:
df.shape

(20601, 4)

In [4]:
df.head()


,0,1,2,3
0,NaN,title,text,label
1,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,1,NaN,Did they post their votes for Hillary already?,1
3,2,UNBELIEVABLE! OBAMAâS ATTORNEY GENERAL SAYS ...,"Now, most of the demonstrators gathered last ...",1
4,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0


In [5]:
df.columns = ['id', 'title', 'text', 'label']


In [6]:
# Drop id column (not useful)
df = df.drop(columns=['id'])




In [7]:
# Handle missing values
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')



In [8]:
# Combine title + text
df['content'] = df['title'] + " " + df['text']



In [9]:
# Separate inputs and labels
X = df['content']
y = df['label']

In [10]:
print(df.columns)
print(y.unique())

Index(['title', 'text', 'label', 'content'], dtype='object')
['label' '1' '0' None ' is just unbelievably rich from North Sea oil. '
 ' Ð¿Ð¾Ñ\x8dÑ\x82Ð¾Ð¼Ñ\x83 Ð² Ñ\x82ÐµÐ¾Ñ\x80Ð¸Ð¸: Ð´Ð¾Ð»Ð¶Ð½Ñ\x8b Ð»Ð¸ Ð½Ð°Ñ\x86Ð¸Ð¾Ð½Ð°Ð»Ñ\x8cÐ½Ñ\x8bÐµ Ð¸Ð·Ð±Ð¸Ñ\x80Ð°Ñ\x82ÐµÐ»Ñ\x8cÐ½Ñ\x8bÐµ Ñ\x81Ð¸Ñ\x81Ñ\x82ÐµÐ¼Ñ\x8b Ñ\x81Ñ\x87Ð¸Ñ\x82Ð°Ñ\x82Ñ\x8cÑ\x81Ñ\x8f ÐºÑ\x80Ð¸Ñ\x82Ð¸Ñ\x87ÐµÑ\x81ÐºÐ¾Ð¹ Ð½Ð°Ñ\x86Ð¸Ð¾Ð½Ð°Ð»Ñ\x8cÐ½Ð¾Ð¹ Ð¸Ð½Ñ\x84Ñ\x80Ð°Ñ\x81Ñ\x82Ñ\x80Ñ\x83ÐºÑ\x82Ñ\x83Ñ\x80Ð¾Ð¹?'
 ' Ð´Ð»Ñ\x8f Ñ\x82Ð¾Ð³Ð¾ Ñ\x87Ñ\x82Ð¾Ð±Ñ\x8b Ð¾Ñ\x82Ð½Ð¾Ñ\x88ÐµÐ½Ð¸Ñ\x8f Ð½Ð¾Ñ\x80Ð¼Ð°Ð»Ð¸Ð·Ð¾Ð²Ð°Ð»Ð¸Ñ\x81Ñ\x8c?'
 ' ÐºÐ¾Ñ\x82Ð¾Ñ\x80Ð¾Ð³Ð¾ Ð¸Ð·Ð±ÐµÑ\x80Ñ\x91Ñ\x82 Ð°Ð¼ÐµÑ\x80Ð¸ÐºÐ°Ð½Ñ\x81ÐºÐ¸Ð¹ Ð½Ð°Ñ\x80Ð¾Ð´ Ð¸ ÐºÐ¾Ñ\x82Ð¾Ñ\x80Ñ\x8bÐ¹ Ð·Ð°Ñ\x85Ð¾Ñ\x87ÐµÑ\x82 Ñ\x80Ð°Ð±Ð¾Ñ\x82Ð°Ñ\x82Ñ\x8c Ñ\x81 Ð½Ð°Ð¼Ð¸.'
 ' Ð±Ñ\x8bÐ»Ð¾ Ð±Ñ\x8b Ð½ÐµÐ»ÐµÐ¿Ñ\x8bÐ¼ Ð½Ðµ Ð¸Ñ\x81Ð¿Ð¾Ð»Ñ\x8cÐ·Ð¾Ð²Ð°Ñ\x82Ñ\x8c Ñ\x81Ð²Ð¾Ñ\x91 Ð³ÐµÐ¾Ð³Ñ\x80Ð°Ñ\x84Ð¸Ñ\x87ÐµÑ\x81ÐºÐ¾Ðµ Ð¿Ñ\x80ÐµÐ¸Ð¼Ñ\x83Ñ\x89ÐµÑ\x81Ñ\x82Ð²Ð¾ Ð¸ Ð½Ðµ Ñ\x80Ð°Ð·Ð²Ð¸Ð²Ð°Ñ\x82Ñ\x8c Ð¾Ñ\x82Ð

In [11]:
df['label'].head(10)

,label
0,label
1,1
2,1
3,1
4,0
5,1
6,1
7,1
8,1
9,1


In [12]:
df['label'] = df['label'].astype(str)

In [13]:
df = df[df['label'].isin(['0', '1'])]


In [14]:
df['label'] = df['label'].astype(int)


In [15]:
print(df['label'].unique())
print(df['label'].dtype)


[1 0]
int64


In [16]:
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

df['content'] = df['title'] + " " + df['text']


In [17]:
X = df['content']
y = df['label']


In [18]:
print(X.shape)
print(y.shape)


(20515,)
(20515,)


In [19]:
from transformers import BertTokenizer


In [20]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [21]:
sample_text = X.iloc[0]
print(sample_text[:200])


LAW ENFORCEMENT ON HIGH ALERT Following Threats Against Cops And Whites On 9-11By #BlackLivesMatter And #FYF911 Terrorists [VIDEO] No comment is expected from Barack Obama Members of the #FYF911 or #F


In [22]:
encoding = tokenizer(
    sample_text,
    max_length=256,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)


In [23]:
encoding.keys()


KeysView({'input_ids': tensor([[  101,  2375,  7285,  2006,  2152,  9499,  2206,  8767,  2114, 10558,
          1998, 12461,  2006,  1023,  1011,  2340,  3762,  1001,  2304,  3669,
          6961, 18900,  3334,  1998,  1001,  1042,  2100,  2546,  2683, 14526,
         15554,  1031,  2678,  1033,  2053,  7615,  2003,  3517,  2013, 13857,
          8112,  2372,  1997,  1996,  1001,  1042,  2100,  2546,  2683, 14526,
          2030,  1001, 11865,  4801, 11253, 17802,  1998,  1001,  2304,  3669,
          6961, 18900,  3334,  5750,  2170,  2005,  1996, 11404,  2075,  1998,
          5689,  1997,  2317,  2111,  1998, 10558,  1012,  2027,  6628,  2500,
          2006,  1037,  2557,  2265,  9857,  2305,  2000,  2735,  1996, 10401,
          1998,  3102,  2317,  2111,  1998, 10558,  2000,  4604,  1037,  4471,
          2055,  1996,  4288,  1997,  2304,  2111,  1999,  2637,  1012,  2028,
          1997,  1996,  1042,  1008,  1008,  1008, 10930, 10258,  8490, 18829,
          2003,  2170,  9609,

In [24]:
encoding['input_ids'].shape
encoding['attention_mask'].shape


torch.Size([1, 256])

In [25]:
encoding['attention_mask']


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [26]:
encodings = tokenizer(
    list(X),
    max_length=256,
    padding=True,
    truncation=True,
    return_tensors='pt'
)


In [27]:
print(encodings['input_ids'].shape)
print(encodings['attention_mask'].shape)
print(y.shape)


torch.Size([20515, 256])
torch.Size([20515, 256])
(20515,)


In [28]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [29]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [31]:
from torch.utils.data import DataLoader
train_dataset = NewsDataset(X_train, y_train, tokenizer)
val_dataset = NewsDataset(X_val, y_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [32]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [35]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)



In [36]:
epochs = 2   # keep small for Colab


In [37]:
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    model.train()

    total_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f"Training loss: {avg_loss:.4f}")



Epoch 1/2
Training loss: 0.0759

Epoch 2/2
Training loss: 0.0242


In [38]:
from sklearn.metrics import accuracy_score
import numpy as np

model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.988057518888618


In [39]:
def predict_news(text):
    model.eval()

    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )

    with torch.no_grad():
        outputs = model(
            input_ids=encoding['input_ids'].to(device),
            attention_mask=encoding['attention_mask'].to(device)
        )

    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "REAL" if prediction == 0 else "FAKE"


In [41]:
user_text = input("Enter a news headline or article:\n")
result = predict_news(user_text)
print("\nPrediction:", result)


Enter a news headline or article:
Breaking Government announces new economic reforms

Prediction: FAKE
